# Sentiment Prediction Model

In [14]:
import sys
sys.path.append('/home/g/code/ds/sentiment')

from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from sentiment_analysis.database import engine
from sentiment_analysis.entities import SigWordUse
from sentiment_analysis.util import rmse

green = '#008f41'
plt.style.use('dark_background')

In [15]:
train, test = onehot.iloc[:,3:], onehot['_rating']
sw = pd.read_sql_table('sig_word', engine)
weights = pd.Series(sw.median_q50.values, index=sw.word).sort_index()
onehot = SigWordUse.onehot()

In [16]:
Qone = train.multiply(weights)
N_total = train.iloc[:,3:].sum(axis=1)
predictions = Qone.sum(axis=1) / N_total
rmse_medians = rmse(test, predictions)
rmse_medians

1.4061896254888429